In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
df = pd.DataFrame()
labels ={'pos':1, 'neg':0}
path = {'./neg', './pos'} # ревью лежат в соответствующих папках, извлеченных из архива
for p in path:
    for fname in os.listdir(p):
        with open(os.path.join(p, fname), 'r') as file:
            text = file.read()
        l = p[-3:] # pos, neg
        df = df.append([[text, labels[l]]], ignore_index=True) # добавляю в датасет ревью и его класс
df.columns = ['review', 'class']
df.head() # отсортированный датасет

,review,class
0,"plot : two teen couples go to a church party ,...",0
1,the happy bastard's quick movie review \ndamn ...,0
2,it is movies like these that make a jaded movi...,0
3,""" quest for camelot "" is warner bros . ' firs...",0
4,synopsis : a mentally unstable man undergoing ...,0


In [4]:
df = df.reindex(np.random.permutation(df.index)) # перемешиваю ревью с помощью индексов строк
# df.head()
df.to_csv('./movie_reviews.csv', index=False) # записываю в csv без индексов
df = pd.read_csv('./movie_reviews.csv')
df.head() # новый перемешанный датасет

,review,class
0,"earth is a harsh , unconsoling drama about the...",1
1,plot : a young man who loves heavy metal music...,1
2,plot : a human space astronaut accidentally fa...,1
3,this movie tries to present itself as the sequ...,0
4,"the central focus of michael winterbottom's "" ...",1


In [5]:
from sklearn.model_selection import train_test_split
X, y = df['review'], df['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
# делю выборку на обучающую и тренировочную в соотнощении 70:30 (1400 и 600 ревью соответственно)

In [6]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

stopwords_nltk = set(stopwords.words("english"))
''' В случае с ревью не очень хорошо исключать абсолютно все стоп-слова, ведь в этом случае некоторые оценочные суждения могут
менять полярность на противоположную: "Not quite sure how to review this film, based on its laughter factor. Well yeah i can,
it's not funny." Если мы исключим отрицание, полярность изменится и наличие слова funny будет свидетельствовать о том,
что ревью положительное, хотя это изначально не так. Поэтому нужно оставить отрицательные слова и смотреть не только на
юниграммы (не знаю, можно ли так сказать), но и на биграммы/триграммы '''
negative_words = set(['isn', 'aren', 'wasn', 'weren', 'ain', 'don', 'not', 'nor', 'but'])
stopwords_filtered = list(stopwords_nltk.difference(negative_words))
# векторизую тексты ревью, исключая только нерелевантные стоп-слова
vectorizer = CountVectorizer(stop_words = stopwords_filtered, max_features = 15000, ngram_range = (1,3))  # много термов-фичей
X_train_features = vectorizer.fit_transform(X_train)
X_test_features = vectorizer.transform(X_test)

In [29]:
from sklearn.metrics import classification_report, f1_score
logistic_model = LogisticRegression() 
logistic_model.fit(X_train_features, y_train)
y_pred = logistic_model.predict(X_test_features)
print(classification_report(y_test, y_pred))
# получилось очень даже хорошо, ф-мера=0.87

             precision    recall  f1-score   support

          0       0.85      0.88      0.87       295
          1       0.88      0.85      0.87       305

avg / total       0.87      0.87      0.87       600



In [30]:
vocabulary = vectorizer.get_feature_names() # имена фичей -- термы
coefs = logistic_model.coef_
word_importances = pd.DataFrame({'word': vocabulary, 'coef': coefs.tolist()[0]}) # сопоставляю термы и коэффициенты логит
word_importances_sorted = word_importances.sort_values(by='coef', ascending = False)
# топ-5 термов, свидетельствующих о том, что ревью -- положительное
word_importances_sorted[:5]

,coef,word
5466,0.556665,fun
11899,0.379303,seen
5965,0.369633,great
14549,0.360070,well
3585,0.351859,different


In [31]:
# топ-5 термов, встречающихся в отрицательных ревью
word_importances_sorted[-6:-1]

,coef,word
13076,-0.471427,supposed
10965,-0.486651,reason
14461,-0.504684,waste
14059,-0.522266,unfortunately
10302,-0.523702,plot


In [32]:
logistic_model.intercept_ # константа 

array([-0.51421409])

In [24]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
scores = []
d = {}
for t in range(60, 100): # ищу оптимальное количество деревьев в rf
    rfc = RandomForestClassifier(n_estimators=t)
    rfc.fit(X_train_features, y_train)
    y_pred = rfc.predict(X_test_features)
    scores.append(f1_score(y_test, y_pred))
    if t not in d:
        d[t] = f1_score(y_test, y_pred)
best_estimator = 70  # пусть будет по умолчанию так
for k, v in d.items():
    if v == max(scores):
        best_estimator = k
print('Best estimators number is', best_estimator)
model = RandomForestClassifier(n_estimators=best_estimator)
model.fit(X_train_features, y_train)
y_pred = model.predict(X_test_features)
print(classification_report(y_test, y_pred)) # ф-мера-0.82, хуже, чем логит

Best estimators number is 93
             precision    recall  f1-score   support

          0       0.80      0.84      0.82       295
          1       0.84      0.80      0.82       305

avg / total       0.82      0.82      0.82       600



In [22]:
# Naive Bayes
from sklearn.naive_bayes import MultinomialNB
naive_model = MultinomialNB()
naive_model.fit(X_train_features, y_train)
y_pred = naive_model.predict(X_test_features)
print(classification_report(y_test, y_pred)) # ф-мера=0.82, хуже, чем логит
# Получилось, что логистическая регрессия лучше всего справилась с задачей

             precision    recall  f1-score   support

          0       0.80      0.86      0.83       295
          1       0.85      0.79      0.82       305

avg / total       0.83      0.82      0.82       600

